# Welcome to Yuna Ai LLM trainer

In [ ]:
!pip install unsloth "xformers==0.0.28.post2"
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install -U peft transformers

# Initialize trainer with paths and hyperparameters (adjust paths as needed)

In [ ]:
from aiflow.trainer import YunaLLMTrainer
trainer = YunaLLMTrainer(
    model_path="yukiarimo/yuna-ai-v4",
    dataset_file="data.jsonl",
    output_dir="newmodel-output",
    max_seq_length=16384,
    load_in_4bit=True,
    device_map={"": 0}
)

# Create model and tokenizer
trainer.create_model(dtype=None)

# Setup LoRA configuration

In [ ]:
trainer.setup_lora(
    r=128,
    lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",
                    "embed_tokens", "lm_head"],
    lora_dropout=0.01,
    bias="all",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=True,
    loftq_config=None
)

# Train the model using UnslothTrainer wrapper

In [ ]:
print("Starting training...")
trainer_stats = trainer.train_model(training_args_kwargs={
    "output_dir": "outputs-pack3",
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 2,
    "max_steps": 300,
    "warmup_steps": 20,
    "warmup_ratio": 0.1,
    "learning_rate": 1e-6,
    "embedding_learning_rate": 1e-7,
    "bf16": True,
    "logging_steps": 1,
    "optim": "adamw_hf",
    "weight_decay": 0.01,
    "lr_scheduler_type": "constant",
    "seed": 42,
    "max_grad_norm": 1.0,
    "group_by_length": True,
    "report_to": "tensorboard",
    "save_steps": 10,
    "gradient_checkpointing": True,
})
print("Trainer stats:", trainer_stats)

# Merge LoRA weights into the base model and save the merged model and tokenizer

In [ ]:
merged_model_dir = trainer.merge_and_save_model(
    merged_model_dir="model-merged",
    checkpoint_dir="checkpoint-20"
)
print("Merged model saved to:", merged_model_dir)

# Upload model to Hugging Face repository

In [ ]:
trainer.huggingface_upload(repo="yukiarimo/yuna-ai-v4-atomic-trained", model_dir="newmodel-outputs"):

# Inference setup: load a model checkpoint for inference

In [ ]:
trainer.inference_setup(
    inference_model_path="/content/himitsu-extra-outputsырше/checkpoint-14",
    max_seq_length=2048,
    dtype=None
)
generated = trainer.generate_text(
    text="<dialog>\n<yuki>How are you?</yuki>\n<yuna>"
)
print("Generated text:\n", generated)

# Create and save new model with updated configuration and additional tokens

In [ ]:
additional_tokens = ["<memory>", "</memory>", "<shujinko>", "</shujinko>",
                        "<kanojo>", "</kanojo>", "<dialog>", "<yuki>", "</yuki>",
                        "<yuna>", "</yuna>", "<hito>", "</hito>", "<qt>", "</qt>",
                        "<action>", "</action>", "<data>", "</data>", "<unk>"]
new_model_dir = trainer.create_and_save_new_model(
    base_model_name="yukiarimo/yuna-ai-v3",
    new_model_dir="yukiarimo/yuna-ai-v4-base",
    temperature=0.7,
    additional_tokens=additional_tokens
)
print("New model created and saved to:", new_model_dir)